In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Convolutional Neural Network
class CNN_Gated_ATSA(nn.Module):
    def __init__(self, args):
        super(CNN_Gated_ATSA, self).__init__()
        self.args = args
        
        # Number of words in vocabulary
        Vocab = args.embed_num
        # Dimension
        Dim = args.embed_dim
        
        # Number of features
        C = args.class_num
        # Number of aspects
        A = args.aspect_num

        # Number of kernels
        K_num = args.kernel_num
        # Size of kernel
        Ks = args.kernel_sizes

        # Creating a Vocab X Dim matrix containing the semantical relations
        self.embed = nn.Embedding(Vocab, Dim)
        # assigning weights to embedding layer using GloVe or Word2Vec
        self.embed.weight = nn.Parameter(args.embedding, requires_grad=True)

        # Creating the embedding matrix for aspects
        self.aspect_embed = nn.Embedding(A, args.aspect_embed_dim)
        # assigning weights to embedding layer using GloVe or Word2Vec
        self.aspect_embed.weight = nn.Parameter(args.aspect_embedding, requires_grad=True)

        # Creating a module list of 1d convolutions applied on kernels
        self.convs1 = nn.ModuleList([nn.Conv1d(D, K_num, K) for K in Ks])
        self.convs2 = nn.ModuleList([nn.Conv1d(D, K_num, K) for K in Ks])
        # A convolutional layer is added to the design for ACSA
        self.convs3 = nn.ModuleList([nn.Conv1d(D, K_num, K, padding=K-2) for K in [3]])

        # makes some of the elements in input tensor zero with 0.2 probability
        self.dropout = nn.Dropout(0.2)

        # y = Wx + b operation
        self.fc1 = nn.Linear(len(Ks)*K_num, C)
        self.fc_aspect = nn.Linear(100, K_num)

    def forward(self, feature, aspect):
        feature = self.embed(feature)  
        aspect_v = self.aspect_embed(aspect)  
        asp1 = [F.relu(conv(aspect_v.transpose(1, 2))) for conv in self.convs3]  
        asp2 = [F.max_pool1d(a, a.size(2)).squeeze(2) for a in asp1]
        aspect_v = torch.cat(asp2, 1)

        x = [F.tanh(conv(feature.transpose(1, 2))) for conv in self.convs1]  
        y = [F.relu(conv(feature.transpose(1, 2)) + self.fc_aspect(aspect_v).unsqueeze(2)) for conv in self.convs2]
        x = [i*j for i, j in zip(x, y)]

        # pooling method
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N,K_num), ...]*len(Ks)

        # concatenating tensors of given dimension (dim=1 in this case)
        x = torch.cat(x, 1)
        x = self.dropout(x)  # (N,len(Ks)*K_num)
        logit = self.fc1(x)  # (N,C)
        return logit, x, y

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=630b80f3-0461-4931-b02f-4187ab638fd0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>